In [1]:
# import openai
import time
import numpy as np
import sys
from copy import deepcopy as dpcp
from datasets import load_dataset
import ast

In [2]:
from transformers import AutoTokenizer, LlamaTokenizer
import transformers
import torch

model = "llama-main/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


2023-11-25 18:36:43.399302: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/just/miniconda3/envs/otdd/lib/
2023-11-25 18:36:43.399579: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/just/miniconda3/envs/otdd/lib/
2023-11-25 18:36:43.399586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Comment: Of course! Based on your interest in "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Sopranos" - This HBO series is a crime drama that explores the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
2. "The Wire" - This HBO series is a gritty and realistic portrayal of the drug trade in Baltimore, exploring the impact of drug addiction and crime on individuals and communities.
3. "Mad Men" - Set in the 1960s, this AMC series follows the lives of advertising executives on Madison Avenue, exploring themes


In [3]:
def get_reply(prompt):
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1000,
    )
    # for seq in sequences:
    #     print(f"Result: {seq['generated_text']}")
        
    return sequences[0]['generated_text']

# SST5

In [9]:
sst5 = load_dataset("SetFit/sst5")

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
len(sst5['train'])

8544

In [13]:
sst5['train'][0]

{'text': 'a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films',
 'label': 4,
 'label_text': 'very positive'}

In [166]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'Given the review:\n'

    n = len(dataset['train'])
    for i in range(examples_num):
        
        rand_i = np.random.randint(n)
        point = dataset['train'][rand_i]
        text = point['text']
        answer = point['label_text']
        pre_prompt += f"Review: {text}\nAnswer: {answer}\n" 
        
    
    test_query = dataset['test'][test_id]['text']
    # pre_prompt += f"Review: {test_query}\nAnswer: "
    pre_prompt += f"{test_query}\n"
    pre_prompt += "Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? \n"

    pre_prompt += 'Rephrase and expand the last review, and provide answer. provide the answer in the format 999 {answer} 999.\n'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer as a python string called chosen.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [167]:
print(construct_prompt(sst5, 0, 0))

Given the review:
no movement , no yuks , not much of anything .
Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? 
Rephrase and expand the last review, and provide answer. provide the answer in the format 999 {answer} 999.



In [186]:
sst5['test'][4]

{'text': 'gangs of new york is an unapologetic mess , whose only saving grace is that it ends by blowing just about everything up .',
 'label': 0,
 'label_text': 'very negative'}

In [187]:
prompt = construct_prompt(sst5, 0, 4)
reply = get_reply(prompt)
print(reply)

Given the review:
gangs of new york is an unapologetic mess , whose only saving grace is that it ends by blowing just about everything up .
Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? 
Rephrase and expand the last review, and provide answer. provide the answer in the format 999 {answer} 999.
The review:
gangs of new york is an unapologetic mess, whose only saving grace is that it ends by blowing just about everything up.
Rephrased and expanded:
The film Gangs of New York is an unabashed disaster, whose only redeeming quality is its explosive finale. 999 {very negative} 999.


In [188]:
reply.split("999")[3]

' {very negative} '

In [189]:
if sst5['test'][4]['label_text'] in reply.split("999")[3]:
    print("Yes:")
else:
    print("N")

Yes:


In [ ]:
total_count = 0
total_correct = 0
for k in range(500):
    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(sst5, 0, k)
    reply = get_reply(prompt)
    # print(reply)
    groundtruth = sst5['test'][k]['label_text']
    if groundtruth in reply.split("999")[3]:
        print("Yes:")
        total_correct += 1
    else:
        print("N")
        print("Reply: ", reply)
        print("GT: ", groundtruth)

k:  0
Yes:
k:  1
Yes:
k:  2
N
Reply:  Given the review:
` how many more voyages can this limping but dearly-loved franchise survive ? '
Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? 
Rephrase and expand the last review, and provide answer. provide the answer in the format 999 {answer} 999.

Answer: 999 {negative} 999.

The review expresses a negative sentiment towards the franchise, indicating that the reviewer believes the franchise is struggling and may not have many more successful voyages. The use of the phrase "limping but dearly-loved" suggests that the reviewer has some affection for the franchise, but also recognizes its current difficulties. Overall, the sentiment of the review is negative, so the answer is 999 {negative} 999.
GT:  neutral
k:  3
N
Reply:  Given the review:
so relentlessly wholesome it made me want to swipe something .
Which of the following sentiment: very negative or negative or neut

In [197]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

146/212 = 0.6886792452830188


# SNLI

In [4]:
snli = load_dataset("snli")

In [5]:
labels = {0: "entailment", 1: "neutral", 2: "contradiction"}

In [6]:
snli

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 550152
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
})

In [304]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant. Given the premise:\n\n'

    n = len(dataset['train'])
#     for i in range(examples_num):
        
#         rand_i = np.random.randint(n)
#         point = dataset['train'][rand_i]
#         text = point['text']
#         answer = point['label_text']
#         pre_prompt += f"Review: {text}\nAnswer: {answer}\n" 
        
    
    test_query = dataset['test'][test_id]['premise']
    test_hypo = dataset['test'][test_id]['hypothesis']
    # pre_prompt += f"Review: {test_query}\nAnswer: "
    pre_prompt += f"{test_query}\n\n"
    pre_prompt += f"and hypothesis\n\n{test_hypo}\n"
    pre_prompt += f'Is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n'
    # pre_prompt += "Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? \n"

    pre_prompt += 'Rephrase and expand the question, and provide answer.\nAnswer:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [305]:
print(construct_prompt(snli, 0, 0))

You are a smart assistant. Given the premise:

This church choir sings to the masses as they sing joyous songs from the book at a church.

and hypothesis

The church has cracks in the ceiling.
Is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?
Rephrase and expand the question, and provide answer.
Answer:


In [308]:
l = 6
prompt = construct_prompt(snli, 0, l)
reply = get_reply(prompt)
print(reply)
print(snli['test'][l])
print(labels[snli['test'][l]['label']])

You are a smart assistant. Given the premise:

An old man with a package poses in front of an advertisement.

and hypothesis

A man poses in front of an ad.
Is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?
Rephrase and expand the question, and provide answer.
Answer: Entailment

Rephrased and expanded question:
Given the premise "An old man with a package poses in front of an advertisement," and the hypothesis "A man poses in front of an ad," does the hypothesis logically follow from the premise?

Explanation:
The hypothesis "A man poses in front of an ad" is a straightforward entailment of the premise "An old man with a package poses in front of an advertisement." The premise provides enough information to logically deduce the hypothesis. Therefore, the correct answer is Entailment.
{'premise': 'An old man with a package poses in front of an advertisement.', 'hypothesis': 'A man poses in front of an ad.', 'label': 0}
entailment


In [268]:
l = 4
prompt = construct_prompt(snli, 0, l)
reply = get_reply(prompt)
print(reply)
print(snli['test'][l])
print(labels[snli['test'][l]['label']])

Given the premise:
A woman with a green headscarf, blue shirt and a very big grin.
and hypothesis
The woman is very happy.
Is the hypothesis "Entailment", "Contradiction", or "Inconclusive" of the premise?
Rephrase and expand the question, and provide answer.
Answer: Inconclusive

Explanation:
The hypothesis "The woman is very happy" is not conclusively supported by the premise "A woman with a green headscarf, blue shirt and a very big grin." The premise provides only limited information about the woman's emotional state, and it is possible that she may be happy, sad, or neutral based on other factors not mentioned in the premise. Therefore, the hypothesis "The woman is very happy" is inconclusive.
{'premise': 'A woman with a green headscarf, blue shirt and a very big grin.', 'hypothesis': 'The woman is very happy.', 'label': 0}
entailment


In [217]:
snli['test'][l]['label']

0

In [218]:
reply.split("@@@")[3]

' Inconclusive '

In [309]:
if labels[snli['test'][l]['label']] in reply.split("Answer:")[1].lower():
    print("Yes:")
else:
    print("N")

Yes:


In [310]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = snli['test'][k]['label']
    
    if label == -1:
        continue
    
    groundtruth = labels[label]

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(snli, 0, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Answer:")
    answer = split_reply[1].strip().lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
N
Reply:  contradiction

rephrased and expanded question:
given the premise "this church choir sings to the masses as they sing joyous songs from the book at a church," and the hypothesis "the church has cracks in the ceiling," is the hypothesis "contradiction" of the premise because the idea of a church choir singing joyous songs contradicts the presence of cracks in the ceiling, which suggests a lack of joy or happiness in the church.
GT:  neutral
0/1 = 0.0
k:  1
Yes:
k:  2
Yes:
k:  3
Yes:
k:  4
Yes:
k:  5
N
Reply:  the hypothesis "entailment" of the premise.

premise: a woman with a green headscarf, blue shirt and a very big grin.

hypothesis: the woman has been shot.

explanation:
the hypothesis "entailment" of the premise is the correct answer because the premise provides no information that would contradict the hypothesis. the premise simply describes a woman with a green headscarf, blue shirt, and a big grin, which does not entail that she has been shot. in fact, it is ent

In [311]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

256/495 = 0.5171717171717172


### Random

In [9]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant.'
    # pre_prompt += f' Given the premise, is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n'
    n = len(dataset['train'])
    for i in range(examples_num):
        
        rand_i = np.random.randint(n)
        point = dataset['train'][rand_i]
        label = point['label']
        while label == -1 or label == '-1':
            rand_i = np.random.randint(n)
            point = dataset['train'][rand_i]
            label = point['label']
        query = point['premise']
        hypo = point['hypothesis']
        answer = labels[point['label']]
        pre_prompt += f"Premise: {query}\nHypothesis: {hypo}\nAnswer: {answer}\n\n" 
        
    
    test_query = dataset['test'][test_id]['premise']
    test_hypo = dataset['test'][test_id]['hypothesis']
    # pre_prompt += f"Review: {test_query}\nAnswer: "
    pre_prompt += f"Premise: {test_query}\n"
    pre_prompt += f"Hypothesis: {test_hypo}\n"
    # pre_prompt += f"Answer:\n"
    pre_prompt += f'Given the premise, is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n'
    pre_prompt += f"Answer:"
    # pre_prompt += "Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? \n"

    # pre_prompt += 'Rephrase and expand the question, and provide answer.\nAnswer:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [15]:
print(construct_prompt(snli, 3, 4))

You are a smart assistant.Premise: A person with dark hair wearing a white tank top, red pants and white shoes is walking down a sidewalk with artwork displayed against a wall.
Hypothesis: A person walks past art that she painted herself.
Answer: neutral

Premise: A man is speaking into a microphone at a podium.
Hypothesis: A woman is speaking into a microphone at a podium.
Answer: contradiction

Premise: Two women stand near a wall plastered with advertisements.
Hypothesis: The women are waiting for a friend and reading the advertisements on the wall
Answer: neutral

Premise: A woman with a green headscarf, blue shirt and a very big grin.
Hypothesis: The woman is very happy.
Given the premise, is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?
Answer:


In [14]:
l = 4
prompt = construct_prompt(snli, 3, l)
reply = get_reply(prompt)
print(reply)
print(snli['test'][l])
print(labels[snli['test'][l]['label']])

You are a smart assistant.Premise: A woman wearing a black dress crosses the street.
Hypothesis: The woman in black was crossed the street.
Answer: entailment

Premise: An very nice capture of a soccer game going on between two men as the man from the team in black attempts to interrupt the red and white teams stride.
Hypothesis: two men playing fifa on xbox on the couch
Answer: contradiction

Premise: A man alone on a gravely beach.
Hypothesis: The man is on a beach.
Answer: entailment

Premise: A woman with a green headscarf, blue shirt and a very big grin.
Hypothesis: The woman is very happy.
Given the premise, is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?
Answer: entailment

Premise: A man in a red shirt and blue jeans is walking down a busy street.
Hypothesis: The man is walking.
Given the premise, is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?
Answer: entailment

Premise: A man wearing a black coat and black pants is 

In [16]:
reply.split("Answer:")

['You are a smart assistant.Premise: A woman wearing a black dress crosses the street.\nHypothesis: The woman in black was crossed the street.\n',
 ' entailment\n\nPremise: An very nice capture of a soccer game going on between two men as the man from the team in black attempts to interrupt the red and white teams stride.\nHypothesis: two men playing fifa on xbox on the couch\n',
 ' contradiction\n\nPremise: A man alone on a gravely beach.\nHypothesis: The man is on a beach.\n',
 ' entailment\n\nPremise: A woman with a green headscarf, blue shirt and a very big grin.\nHypothesis: The woman is very happy.\nGiven the premise, is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n',
 ' entailment\n\nPremise: A man in a red shirt and blue jeans is walking down a busy street.\nHypothesis: The man is walking.\nGiven the premise, is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n',
 ' entailment\n\nPremise: A man wearing a black coat and b

In [20]:
reply.split("Answer:")[4].strip().split(" ")[0]

'entailment\n\nPremise:'

In [ ]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = snli['test'][k]['label']
    
    if label == -1:
        continue
    
    groundtruth = labels[label]

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(snli, 3, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Answer:")
    answer = split_reply[4].strip().split(" ")[0].lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
N
Reply:  entailment
GT:  neutral
0/1 = 0.0
k:  1
Yes:
k:  2
N
Reply:  entailment

premise:
GT:  contradiction
k:  3
N
Reply:  entailment

premise:
GT:  neutral
k:  4
Yes:
k:  5
N
Reply:  entailment
GT:  contradiction
k:  6
Yes:
k:  7
N
Reply:  entailment
GT:  neutral
k:  8
N
Reply:  entailment

premise:
GT:  contradiction
k:  9
Yes:
k:  10
N
Reply:  neutral

premise:
GT:  entailment
4/11 = 0.36363636363636365
k:  11
N
Reply:  entailment

premise:
GT:  contradiction
k:  12
Yes:
k:  13
Yes:
k:  14
Yes:
k:  15
N
Reply:  entailment

premise:
GT:  contradiction
k:  16
Yes:
k:  17
N
Reply:  entailment
GT:  neutral
k:  18
Yes:
k:  19
N
Reply:  entailment
GT:  contradiction
k:  20
N
Reply:  entailment

premise:
GT:  neutral
9/21 = 0.42857142857142855
k:  21
N
Reply:  entailment

premise:
GT:  contradiction
k:  22
Yes:
k:  23
N
Reply:  entailment
GT:  neutral
k:  24
N
Reply:  entailment
GT:  neutral
k:  25
N
Reply:  neutral

premise:
GT:  contradiction
k:  26
Yes:
k:  27
Yes:
k:  28
Yes:

In [25]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

188/495 = 0.3797979797979798


# Yelp

In [28]:
yelp = load_dataset("KaiLv/UDR_Yelp")

In [313]:
yelp

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 30000
    })
    test: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 3000
    })
    debug: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 5000
    })
})

In [324]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant. Given the review:\n\n'

    n = len(dataset['train'])
#     for i in range(examples_num):
        
#         rand_i = np.random.randint(n)
#         point = dataset['train'][rand_i]
#         text = point['text']
#         answer = point['label_text']
#         pre_prompt += f"Review: {text}\nAnswer: {answer}\n" 
        
    
    test_query = dataset['test'][test_id]['sentence']
    # pre_prompt += f"Review: {test_query}\nAnswer: "
    pre_prompt += f"{test_query}\n\n"
    
    pre_prompt += f'What is the rating of the review from scores 1 (most negative), 2, 3, 4, or 5 (most positive)?\n'
    # pre_prompt += "Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? \n"

    pre_prompt += 'Rephrase and expand the review, and provide answer.\nRating:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [325]:
print(construct_prompt(yelp, 0, 0))

You are a smart assistant. Given the review:

Excellent sushi, great service, and good happy hour specials.  It has a decent sports bar theme.  I had my bachelor party here and even my friends who don't care for fish liked the place.  I also go here for business dinners.\n\nTry the Ladybug Roll just for the presentation.

What is the rating of the review from scores 1 (most negative), 2, 3, 4, or 5 (most positive)?
Rephrase and expand the review, and provide answer.
Rating:


In [326]:
l = 6
prompt = construct_prompt(yelp, 0, l)
reply = get_reply(prompt)
print(reply)
print(yelp['test'][l])
# print(labels[snli['test'][l]['label']])

You are a smart assistant. Given the review:

My only complaint, truly, was that I found it to be a bit loud and so, once we moved to a sofa seating arrangement, I had a hard time hearing my friends who were sitting across from me. The music was great, just a tad too much on the side of I-am-not-involved-in-the-convo-because-I-have-no-idea-what-the-convo-is.\n\nAll things considered, however, it was fairly minor. At the bar, it wasn't as bad. Maybe that's the key? OK, got that bad news out of the way? Great, moving on.\n\nWe visited level 1.5 twice in one night... I am not sure what that tells you, but I can think of a few things. ;)\n\nThe service was superb (thanks, Brandy!), the drinks were strong (loved the slightly sweet, slightly bitter London Summer and the sweet margarita shot), and the other patrons were awesome. This level envelopes you in the chandelier idea and it's easy to forget  - supported by the amazing service and beautiful draped lighting - that you are in a casino. 

In [327]:
reply.split("Rating:")

['You are a smart assistant. Given the review:\n\nMy only complaint, truly, was that I found it to be a bit loud and so, once we moved to a sofa seating arrangement, I had a hard time hearing my friends who were sitting across from me. The music was great, just a tad too much on the side of I-am-not-involved-in-the-convo-because-I-have-no-idea-what-the-convo-is.\\n\\nAll things considered, however, it was fairly minor. At the bar, it wasn\'t as bad. Maybe that\'s the key? OK, got that bad news out of the way? Great, moving on.\\n\\nWe visited level 1.5 twice in one night... I am not sure what that tells you, but I can think of a few things. ;)\\n\\nThe service was superb (thanks, Brandy!), the drinks were strong (loved the slightly sweet, slightly bitter London Summer and the sweet margarita shot), and the other patrons were awesome. This level envelopes you in the chandelier idea and it\'s easy to forget  - supported by the amazing service and beautiful draped lighting - that you are 

In [328]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = yelp['test'][k]['label']
    
    if label == -1:
        continue
    
    groundtruth = str(label+1)

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(yelp, 0, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Rating:")
    answer = split_reply[1].strip().lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
Yes:
1/1 = 1.0
k:  1
Yes:
k:  2
Yes:
k:  3
Yes:
k:  4
N
Reply:  4

the reviewer enjoyed their meal at this restaurant, but had a few issues. they were disappointed that their vegan friend was unable to get a basic vegetable wrap due to kitchen inconvenience, and that the server did not offer any alternatives. however, the reviewer was pleased with the hummus, bread, and cheese curds. they also enjoyed their crispy chicken sandwich with guacamole. the reviewer notes that while this restaurant is not the best for burgers, it is a great spot for trivia nights or watching a game. overall, the reviewer gives the restaurant a rating of 4 out of 5.
GT:  3
k:  5
Yes:
k:  6
Yes:
k:  7
Yes:
k:  8
N
Reply:  1 (most negative)

the reviewer had high expectations for this restaurant based on its menu and ambiance, but unfortunately, their experience was a complete disaster. the place was chaotic, with servers, food runners, and bussers running into each other, and the cleanliness was subpar, w

In [330]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

347/500 = 0.694


In [331]:
347/500

0.694

## Random

In [39]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant. \n'

    n = len(dataset['train'])
    for i in range(examples_num):
        
        rand_i = np.random.randint(n)
        point = dataset['train'][rand_i]
        text = point['sentence']
        answer = int(point['label'])+1
        pre_prompt += f"Review: {text}\nRating: {answer}\n" 
        
    
    test_query = dataset['test'][test_id]['sentence']
    # pre_prompt += f"Review: {test_query}\nAnswer: "
    pre_prompt += f"Review:{test_query}\n"
    
    pre_prompt += f'Given the review, what is the rating of the review from scores 1 (most negative), 2, 3, 4, or 5 (most positive)?\n'
    # pre_prompt += "Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? \n"

    pre_prompt += 'Rating:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [40]:
print(construct_prompt(yelp, 3, 0))

You are a smart assistant. 
Review: If it weren't for the excellent service this place would have 1 star. The food was awful, the soda was watered down, and the restrooms stunk like sewer. I have eaten here years ago and it was very good, i'm very very disappointed in Bonito Michoacan. Also, the mariachi band, AWFUL and ANNOYING!\n\nNote: try Frank and Finas on Grand Canyon and Flamingo instead, best Mexican food in town, IMO. And guess what, no mariachi band:)
Rating: 2
Review: !  We had a 50 th B-day party past week-end, 53 guests all exclaiming how good the  food was, accents on the tuna tartar and filet mignon cooked to perfection that melted in your mouth. The service was fantastic, always a refill !!!  From the sliders to the lamb chops served on the terrace as our guests arrived followed by constant clearing of empty glasses and napkins. I would recommend Otto for any function or a delicious dinner.. Very honest gentleman
Rating: 4
Review: a bit pricy but really good and the que

In [41]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = yelp['test'][k]['label']
    
    if label == -1:
        continue
    
    groundtruth = str(label+1)

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(yelp, 3, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Rating:")
    answer = split_reply[4].strip().split(" ")[0].lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
N
Reply:  1
GT:  5
0/1 = 0.0
k:  1


/home/just/miniconda3/envs/tiger3/lib/python3.9/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 1376, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


N
Reply:  
GT:  5
k:  2
N
Reply:  
GT:  1
k:  3
N
Reply:  
GT:  5
k:  4
N
Reply:  2
GT:  3
k:  5
N
Reply:  1

note:
GT:  2
k:  6


/home/just/miniconda3/envs/tiger3/lib/python3.9/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 1138, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


N
Reply:  
GT:  4
k:  7
Yes:
k:  8
N
Reply:  1
GT:  2
k:  9
N
Reply:  2

answer:
GT:  5
k:  10
N
Reply:  2
GT:  1
1/11 = 0.09090909090909091
k:  11


...

N
Reply:  
GT:  5
121/481 = 0.2515592515592516
k:  481
Yes:
k:  482
N
Reply:  1

answer:
GT:  3
k:  483
N
Reply:  3
GT:  5
k:  484
Yes:
k:  485
N
Reply:  1
explanation:
GT:  4
k:  486
N
Reply:  
GT:  2
k:  487
Yes:
k:  488
N
Reply:  1
GT:  3
k:  489
N
Reply:  1
GT:  2
k:  490
N
Reply:  1
GT:  3
124/491 = 0.2525458248472505
k:  491
Yes:
k:  492


/home/just/miniconda3/envs/tiger3/lib/python3.9/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 1181, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


N
Reply:  
GT:  4
k:  493
N
Reply:  3
GT:  4
k:  494
N
Reply:  1
GT:  4
k:  495
Yes:
k:  496
N
Reply:  1
GT:  2
k:  497


/home/just/miniconda3/envs/tiger3/lib/python3.9/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 2038, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


N
Reply:  
GT:  2
k:  498
Yes:
k:  499
N
Reply:  1
GT:  5


In [37]:
print(reply)

You are a smart assistant. 
Review:Excellent sushi, great service, and good happy hour specials.  It has a decent sports bar theme.  I had my bachelor party here and even my friends who don't care for fish liked the place.  I also go here for business dinners.\n\nTry the Ladybug Roll just for the presentation.
Given the review, what is the rating of the review from scores 1 (most negative), 2, 3, 4, or 5 (most positive)?
Rating: 5 (most positive)
The reviewer mentions several positive aspects of their experience, including the excellent sushi, great service, good happy hour specials, and a decent sports bar theme. They also mention that even their friends who don't care for fish enjoyed the place, and that they go there for business dinners. These comments suggest a very positive overall experience, earning a rating of 5 (most positive).


In [42]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

127/500 = 0.254


# ComE

In [45]:
come = load_dataset("KaiLv/UDR_ComE")

In [46]:
come

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'question', 'choices', 'len_question', 'max_len_choices'],
        num_rows: 9996
    })
    test: Dataset({
        features: ['idx', 'label', 'question', 'choices', 'len_question', 'max_len_choices'],
        num_rows: 1000
    })
    debug: Dataset({
        features: ['idx', 'label', 'question', 'choices', 'len_question', 'max_len_choices'],
        num_rows: 5000
    })
})

In [47]:
def get_choices(choices):
    
    first = choices.split("B.")
    answer_a = first[0].strip().split("A.")[1].strip()
    second = first[1].split("C.")
    answer_b = second[0].strip()
    answer_c = second[1].strip()
    
    return [answer_a, answer_b, answer_c]

In [48]:
len("Select the most corresponding reason why this statement is against common sense.")

80

In [49]:
def get_question(question):
    
    bef_opt = question.split("Options:")[0]
    after_int = bef_opt[80:].strip()
    
    return after_int

In [50]:
def get_label(label):
    labels = {'A': '1', 'B': '2', 'C': '3', 'D': '4'}
    let_lab = label[0]
    
    return labels[let_lab]

In [51]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant. Given the sentence that is against common sense:\n\n'

    n = len(dataset['train'])
#     for i in range(examples_num):
        
#         rand_i = np.random.randint(n)
#         point = dataset['train'][rand_i]
#         text = point['text']
#         answer = point['label_text']
#         pre_prompt += f"Review: {text}\nAnswer: {answer}\n" 
        
    
    test_query = get_question(dataset['test'][test_id]['question'])
    
    choices = get_choices(dataset['test'][test_id]['choices'])
    
    
    
    
    # test_hypo = dataset['test'][test_id]['hypothesis']
    pre_prompt += f"{test_query}\n\n"
    
    pre_prompt += "Select a reason why the sentence is against common sense.\nReasons:\n"
    
    for i in range(len(choices)):
        pre_prompt += f"{i+1}. {choices[i]}\n"
        
    # pre_prompt += f'Is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n'
    # pre_prompt += "Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? \n"

    pre_prompt += 'Rephrase and expand the question, and provide answer.\nReason:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [451]:
print(construct_prompt(come, 0, 0))


You are a smart assistant. Given the sentence that is against common sense:

He loves to stroll at the park with his bed

Select a reason why the sentence is against common sense.
Reasons:
1. A bed is too heavy to carry with when strolling at a park
2. walking at a park is good for health
3. Some beds are big while some are smaller
Rephrase and expand the question, and provide answer.
Reason:


In [438]:
get_question(come['test'][11]['question'])

'crack addicts are addicted to chocolate milk'

In [430]:
get_choices(come['test'][11]['choices'])

['chocolate milk is delicious to drink on hot',
 'you cannot inject chocolate milk',
 'chocolate milk is not addictive']

In [ ]:
get

In [452]:
l = 0
prompt = construct_prompt(come, 0, l)
reply = get_reply(prompt)
print(reply)
print(come['test'][l])
print(get_label(come['test'][l]['label']))

You are a smart assistant. Given the sentence that is against common sense:

He loves to stroll at the park with his bed

Select a reason why the sentence is against common sense.
Reasons:
1. A bed is too heavy to carry with when strolling at a park
2. walking at a park is good for health
3. Some beds are big while some are smaller
Rephrase and expand the question, and provide answer.
Reason: 1

Please select the reason why the sentence is against common sense and provide an answer.
{'idx': 0, 'label': 'A. A bed is too heavy to carry with when strolling at a park', 'question': 'Select the most corresponding reason why this statement is against common sense. He loves to stroll at the park with his bed Options: A. A bed is too heavy to carry with when strolling at a park B. walking at a park is good for health C. Some beds are big while some are smaller', 'choices': 'A. A bed is too heavy to carry with when strolling at a park\nB. walking at a park is good for health\nC. Some beds are bi

In [396]:
come['test'][l]['label'] in reply.split("Answer:")[1]

False

In [ ]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = get_label(come['test'][k]['label'])
    
    if label == -1:
        continue
    
    groundtruth = label

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(come, 0, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Reason:")
    answer = split_reply[1].strip().lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
Yes:
1/1 = 1.0
k:  1
N
Reply:  1. an inverter is smaller than a car.

please select the reason why the sentence is against common sense.
GT:  2
k:  2
N
Reply:  2
explanation: the sentence is against common sense because lemons are not typically a pizza topping. pizzas are usually topped with ingredients such as cheese, vegetables, meats, and sauces, but lemons are not one of them. therefore, it is unlikely that the chef would put extra lemons on the pizza.
GT:  3
k:  3
Yes:
k:  4
N
Reply:  1. a plane flies upon the garden.

answer:
the sentence is against common sense because planes do not typically fly in gardens. gardens are typically land-based environments, and planes are aircraft that fly in the air. it is unlikely that you would find a plane in a garden, as the two are not compatible environments. therefore, the sentence is against common sense.
GT:  3
k:  5
Yes:
k:  6
N
Reply:  the sentence is against common sense because not everybody needs to pay taxes.

please provide a

In [455]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

296/500 = 0.592


## Random

In [54]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant.\n'

    n = len(dataset['train'])
    for i in range(examples_num):
        
        rand_i = np.random.randint(n)
        point = dataset['train'][rand_i]
        
        tr_query = get_question(point['question'])
        tr_choices = get_choices(point['choices'])
        tr_answer = get_label(point['label'])
        
        

        pre_prompt += f"Sentence: {tr_query}\nReasons:\n" 
        for i in range(len(tr_choices)):
            pre_prompt += f"{i+1}. {tr_choices[i]}\n"
        pre_prompt += f"Reason: {tr_answer}\n\n"
    
    
    test_query = get_question(dataset['test'][test_id]['question'])
    
    choices = get_choices(dataset['test'][test_id]['choices'])
    
    # test_hypo = dataset['test'][test_id]['hypothesis']
    pre_prompt += f"Sentence: {test_query}\nReasons:\n"
    for i in range(len(choices)):
        pre_prompt += f"{i+1}. {choices[i]}\n"
        
    pre_prompt += "Given the sentence that is against common sense, select a reason why the sentence is against common sense.\n"
    

        
    # pre_prompt += f'Is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n'
    # pre_prompt += "Which of the following sentiment: very negative or negative or neutral or positive or very positive most suits the review? \n"

    pre_prompt += 'Reason:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [55]:
print(construct_prompt(come, 3, 0))


You are a smart assistant.
Sentence: Eating too much for supper is good for health.
Reasons:
1. People should have good eating habits because it's god for health.
2. Eating too late for supper is not good for health.
3. Eating too much supper can cause a burden on the stomach.
Reason: 3

Sentence: a motorway is a place for bikes to travel on
Reasons:
1. Bikes are not automobile while a motorway is for automobile vehicles
2. Bikes only have two wheels and they are too slow to travel on on motorway
3. A motorway is too long for riders to ride through it without getting exhausted
Reason: 1

Sentence: he prevented the car from going on the road
Reasons:
1. he found a lot of traffic on the road so he decided not to drive
2. the road is for cars to go on
3. there's something wrong with the car so he pulls over it
Reason: 2

Sentence: He loves to stroll at the park with his bed
Reasons:
1. A bed is too heavy to carry with when strolling at a park
2. walking at a park is good for health
3. Som

In [57]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = get_label(come['test'][k]['label'])
    
    if label == -1:
        continue
    
    groundtruth = label

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(come, 3, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Reason:")
    answer = split_reply[4].strip().split(" ")[0].lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
N
Reply:  2

answer:
the
GT:  1
0/1 = 0.0
k:  1
Yes:
k:  2
N
Reply:  2

sentence:
GT:  3
k:  3
N
Reply:  2

answer:
GT:  3
k:  4
N
Reply:  2

answer:
GT:  3
k:  5
N
Reply:  3

answer:
GT:  2
k:  6
N
Reply:  1
GT:  2
k:  7
Yes:
k:  8
Yes:
k:  9
N
Reply:  2
GT:  3
k:  10
N
Reply:  1

please
GT:  3
3/11 = 0.2727272727272727
k:  11
Yes:
k:  12
N
Reply:  3

the
GT:  1
k:  13
N
Reply:  2

sentence:
GT:  3
k:  14
Yes:
k:  15
N
Reply:  3

please
GT:  1
k:  16
N
Reply:  2

answer:

sentence:
GT:  1
k:  17
N
Reply:  1
GT:  3
k:  18
N
Reply:  3
GT:  1
k:  19
Yes:
k:  20
N
Reply:  1
GT:  2
6/21 = 0.2857142857142857
k:  21
Yes:
k:  22
Yes:
k:  23
N
Reply:  3
GT:  1
k:  24
Yes:
k:  25
Yes:
k:  26
Yes:
k:  27
Yes:
k:  28
Yes:
k:  29
N
Reply:  2

sentence:
GT:  1
k:  30
N
Reply:  2

sentence:
GT:  1
13/31 = 0.41935483870967744
k:  31
N
Reply:  1
GT:  2
k:  32
N
Reply:  2

answer:
GT:  3
k:  33
N
Reply:  2
GT:  3
k:  34
N
Reply:  2
GT:  1
k:  35
Yes:
k:  36
N
Reply:  2

sentence:
GT:  3
k:  37
Ye

In [58]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

227/500 = 0.454


# ARC-Challenge

In [100]:
arc = load_dataset("ai2_arc", 'ARC-Challenge')

In [101]:
arc

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 1119
    })
    test: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 1172
    })
    validation: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 299
    })
})

In [350]:
arc['test'][0]

{'id': 'Mercury_7175875',
 'question': 'An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?',
 'choices': {'text': ['Planetary density will decrease.',
   'Planetary years will become longer.',
   'Planetary days will become shorter.',
   'Planetary gravity will become stronger.'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'C'}

In [356]:
len(arc['test'][0]['choices']['text'])

4

In [102]:
labels = {'A': "1", 'B': "2", 'C': "3", 'D': "4", '1':'1', '2':'2', '3':'3', '4':'4'}

In [103]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant. Given the statement:\n\n'

    n = len(dataset['train'])
#     for i in range(examples_num):
        
#         rand_i = np.random.randint(n)
#         point = dataset['train'][rand_i]
#         text = point['text']
#         answer = point['label_text']
#         pre_prompt += f"Review: {text}\nAnswer: {answer}\n" 
        
    
    test_query = dataset['test'][test_id]['question']
    pre_prompt += f"{test_query}\nOptions:\n"
    
    choices = dataset['test'][test_id]['choices']['text']
    labels = dataset['test'][test_id]['choices']['label']
    for i in range(len(choices)):
        pre_prompt += f"{i+1}. {choices[i]}\n"
    # pre_prompt += f'Is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n'
    pre_prompt += "Which of the options best answers the question?\n"

    pre_prompt += 'Rephrase and expand the statement, and provide answer.\nAnswer:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [104]:
print(construct_prompt(arc, 0, 0))


You are a smart assistant. Given the statement:

An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?
Options:
1. Planetary density will decrease.
2. Planetary years will become longer.
3. Planetary days will become shorter.
4. Planetary gravity will become stronger.
Which of the options best answers the question?
Rephrase and expand the statement, and provide answer.
Answer:


In [105]:
l = 0
prompt = construct_prompt(arc, 0, l)
reply = get_reply(prompt)
print(reply)
print(arc['test'][l])
# print(labels[snli['test'][l]['label']])

You are a smart assistant. Given the statement:

An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?
Options:
1. Planetary density will decrease.
2. Planetary years will become longer.
3. Planetary days will become shorter.
4. Planetary gravity will become stronger.
Which of the options best answers the question?
Rephrase and expand the statement, and provide answer.
Answer: 3. Planetary days will become shorter.
Explanation: The increase in rotation of a planet after a meteorite impact will result in a shorter day. This is because the planet's rotation period will decrease due to the increased angular momentum. As the planet rotates faster, its day will become shorter. Therefore, option 3 is the most likely effect of this increase in rotation.
{'id': 'Mercury_7175875', 'question': 'An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this 

In [106]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = arc['test'][k]['answerKey']
    
    if label == -1:
        continue
    
    groundtruth = labels[label]

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(arc, 0, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Answer:")
    answer = split_reply[1].strip().lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
Yes:
1/1 = 1.0
k:  1
Yes:
k:  2
N
Reply:  2. light energy is converted to chemical energy.

explanation:
photosynthesis is the process by which light energy is converted into chemical energy in the form of glucose and oxygen. this process begins with the absorption of light energy by pigments such as chlorophyll, which are present in the leaves of plants. the light energy is then converted into chemical energy through a series of reactions involving the conversion of atp and nadph. these chemical energy molecules are then used to convert carbon dioxide into glucose, which is the final product of photosynthesis. therefore, option 2 is the correct answer.
GT:  3
k:  3
N
Reply:  3. the dependent (responding) variable.
explanation:
the physicist is investigating the dependent variable, which is the speed at which the car must reach to jump over the ramp. the physicist is changing the independent variable, which is the speed of the car, in trials two and three, and observing the respo

In [107]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

360/500 = 0.72


In [370]:
arc['test'][k]

{'id': 'NYSEDREGENTS_2008_8_34',
 'question': 'Which statement best describes the energy changes that occur while a child is riding on a sled down a steep, snow-covered hill?',
 'choices': {'text': ['Kinetic energy decreases and potential energy increases.',
   'Kinetic energy increases and potential energy decreases.',
   'Both potential energy and kinetic energy decrease.',
   'Both potential energy and kinetic energy increase.'],
  'label': ['1', '2', '3', '4']},
 'answerKey': '2'}

## Random

In [80]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant. \n\n'

    n = len(dataset['train'])
    for i in range(examples_num):
        
        rand_i = np.random.randint(n)
        point = dataset['train'][rand_i] 
        label = point['answerKey']
    
        while label == -1 or label == "-1":
            rand_i = np.random.randint(n)
            point = dataset['train'][rand_i]
            label = point['answerKey']

        label_str = labels[label]
        
        tr_query = point['question']
        tr_choices = point['choices']['text']
        tr_clabels = point['choices']['label']
        
        pre_prompt += f"{tr_query}\nOptions:\n"
        for i in range(len(tr_choices)):
            pre_prompt += f"{i+1}. {tr_choices[i]}\n"
        pre_prompt += f"Answer: {label_str}\n" 
        
    test_query = dataset['test'][test_id]['question']
    pre_prompt += f"{test_query}\nOptions:\n"
    
    choices = dataset['test'][test_id]['choices']['text']

    for i in range(len(choices)):
        pre_prompt += f"{i+1}. {choices[i]}\n"
    # pre_prompt += f'Is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n'
        pre_prompt += "Given the context and question, which of the options best answers the question?\n"

    pre_prompt += 'Answer:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [81]:
print(construct_prompt(arc, 3, 0))

You are a smart assistant. 

Which of the following is a trait that a dog does NOT inherit from its parents?
Options:
1. the length of its fur
2. the shape of its nose
3. the size of its appetite
4. the color of its fur
Answer: 3
All of the following are examples of erosion EXCEPT:
Options:
1. The wind in the desert blows sand against a rock.
2. A glacier picks up boulders as it moves.
3. A flood washes over a riverbank, and the water carries small soil particles downstream.
4. An icy winter causes the pavement in a road to crack.
Answer: 4
Why is it winter in North America when it is summer in South America?
Options:
1. The south is always warmer than the north.
2. There is less land than water in the south.
3. North America receives less direct sunlight during the winter.
4. When it is December in North America, it is June in South America.
Answer: 3
An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotat

In [90]:
l = 0
prompt = construct_prompt(arc, 3, 6)
reply = get_reply(prompt)
print(reply)
print(arc['test'][l])
# print(labels[snli['test'][l]['label']])

You are a smart assistant. 

If there is a great deal of precipitation occurring in a short time, which process in the water cycle will be affected first?
Options:
1. runoff
2. evaporation
3. condensation
4. transpiration
Answer: 1
Which weather event usually includes heavy precipitation, strong winds, and surface air temperatures below 0°C?
Options:
1. blizzard
2. hurricane
3. thunderstorm
4. tornado
Answer: 1
In addition to oxygen, what do plants produce during photosynthesis?
Options:
1. chlorophyll
2. sugar
3. carbon dioxide
4. light energy
Answer: 2
A type of small mammal from the mountain regions of the western United States makes its home out of piles of rock. During summer months, the mammal places grasses and seeds in protected places in the rock piles. Which of the following is the most likely reason for this behavior?
Options:
1. to repare for migration before winter
Given the context and question, which of the options best answers the question?
2. to provide warmth during t

In [92]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = arc['test'][k]['answerKey']
    
    if label == -1:
        continue
    
    groundtruth = labels[label]

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(arc, 3, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Answer:")
    answer = split_reply[4].strip().split(" ")[0].lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
N
Reply:  2
GT:  3
0/1 = 0.0
k:  1
Yes:
k:  2
N
Reply:  2
GT:  3
k:  3
Yes:
k:  4
N
Reply:  2
GT:  4
k:  5
Yes:
k:  6
Yes:
k:  7
N
Reply:  1
GT:  3
k:  8
N
Reply:  1
which
GT:  2
k:  9
N
Reply:  2
what
GT:  1
k:  10
N
Reply:  3
what
GT:  2
4/11 = 0.36363636363636365
k:  11
N
Reply:  3
GT:  1
k:  12
N
Reply:  2
the
GT:  3
k:  13
N
Reply:  4
a
GT:  3
k:  14
N
Reply:  4
GT:  3
k:  15
N
Reply:  2
GT:  1
k:  16
Yes:
k:  17
N
Reply:  1
GT:  2
k:  18
N
Reply:  2
GT:  1
k:  19
N
Reply:  2
what
GT:  3
k:  20
Yes:
6/21 = 0.2857142857142857
k:  21
Yes:
k:  22
Yes:
k:  23
N
Reply:  2
GT:  3
k:  24
N
Reply:  4
a
GT:  2
k:  25
N
Reply:  2
the
GT:  4
k:  26
Yes:
k:  27
Yes:
k:  28
Yes:
k:  29
Yes:
k:  30
Yes:
13/31 = 0.41935483870967744
k:  31
Yes:
k:  32
Yes:
k:  33
Yes:
k:  34
N
Reply:  2
which
GT:  1
k:  35
N
Reply:  2
which
GT:  1
k:  36
Yes:
k:  37
Yes:
k:  38
Yes:
k:  39
Yes:
k:  40
Yes:
21/41 = 0.5121951219512195
k:  41
N
Reply:  2
which
GT:  1
k:  42
Yes:
k:  43
Yes:
k:  44
N
Reply:  1


In [93]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

256/500 = 0.512


# Cosmos QA

In [94]:
cqa = load_dataset("cosmos_qa")

In [378]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant. Given the context:\n\n'

    n = len(dataset['train'])
#     for i in range(examples_num):
        
#         rand_i = np.random.randint(n)
#         point = dataset['train'][rand_i]
#         text = point['text']
#         answer = point['label_text']
#         pre_prompt += f"Review: {text}\nAnswer: {answer}\n" 
        
    
    test_context = dataset['validation'][test_id]['context']
    test_query = dataset['validation'][test_id]['question']
    pre_prompt += f"{test_context}\n\n{test_query}\nOptions:\n"
    
    # choices = dataset['validation'][test_id]['choices']['text']
    # labels = dataset['validation'][test_id]['choices']['label']
    for i in range(4):
        answer_i = dataset['validation'][test_id][f'answer{i}']
        pre_prompt += f"{i+1}. {answer_i}\n"
    # pre_prompt += f'Is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n'
    # pre_prompt += "Which of the options explains that the statement is against common sense?\n"

    pre_prompt += 'Rephrase and expand the question, and provide answer.\nAnswer:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [382]:
l = 2
prompt = construct_prompt(cqa, 0, l)
reply = get_reply(prompt)
print(reply)
print(cqa['validation'][l])
# print(labels[snli['test'][l]['label']])

You are a smart assistant. Given the context:

I was walking home from the store , when I saw an old man laying on the sidewalk , bleeding . The right side of his face was all covered in blood . He was conscious but seemed dazed and probably intoxicated . Nearby there was a young man dialing his cell phone .

What may happen after the young man makes his call ?
Options:
1. An ambulance would likely come to the scene
2. The taxi would pick up the young man
3. None of the above choices .
4. The bus would arrive at the stop soon
Rephrase and expand the question, and provide answer.
Answer:
1. An ambulance would likely come to the scene.

Explanation:
The young man's call to 911 or the local emergency number would have alerted the authorities to the situation. An ambulance would likely be dispatched to the location to provide medical attention to the injured old man. This is the most likely outcome given the information provided in the scenario.
{'id': '3AXFSPQOYQW4GFJD8K4D3JADN54FJ9##352Y

In [383]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = cqa['validation'][k]['label']
    
    if label == -1:
        continue
    
    groundtruth = str(label+1)

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(cqa, 0, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Answer:")
    answer = split_reply[1].strip().lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
Yes:
1/1 = 1.0
k:  1
N
Reply:  you are concerned about getting married to a woman who is not of the same religion as you. you are worried that if the woman you were previously married to comes back, she might cause problems for the girl you are planning to marry. however, legally, there is no compulsion to get a divorce before remarrying. you can register your marriage with the girl you want to marry, but it is important to understand that polygamy is illegal in many countries and can lead to legal complications. it is essential to consider the laws and cultural norms of the country you are in before making any decisions.

please let me know if you have any other questions.
GT:  1
k:  2
Yes:
k:  3
Yes:
k:  4
Yes:
k:  5
N
Reply:  4. he was waiting on a ride.

explanation:
the old man was likely waiting for a ride, possibly from a friend or family member, as he was laying on the sidewalk and appeared to be dazed and intoxicated. the nearby young man was dialing his cell phone, indi

In [384]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

249/500 = 0.498


## Random

In [95]:
def construct_prompt(dataset, examples_num, test_id):
    pre_prompt = 'You are a smart assistant. \n'

    n = len(dataset['train'])
    for i in range(examples_num):
        
        rand_i = np.random.randint(n)
        point = dataset['train'][rand_i]
        answer = int(point['label'])+1
        
        while answer == -1 or answer == "-1":
            rand_i = np.random.randint(n)
            point = dataset['train'][rand_i]
            answer = int(point['label'])+1
        
        context = point['context']
        query = point['question']
        
        pre_prompt += f"{context}\n\n{query}\nOptions:\n"
        for i in range(4):
            answer_i = point[f'answer{i}']
            pre_prompt += f"{i+1}. {answer_i}\n"
        pre_prompt += f"Answer: {answer}\n\n"
        
    test_context = dataset['validation'][test_id]['context']
    test_query = dataset['validation'][test_id]['question']
    pre_prompt += f"{test_context}\n{test_query}\nOptions:\n"
    
    # choices = dataset['validation'][test_id]['choices']['text']
    # labels = dataset['validation'][test_id]['choices']['label']
    for i in range(4):
        answer_i = dataset['validation'][test_id][f'answer{i}']
        pre_prompt += f"{i+1}. {answer_i}\n"
    # pre_prompt += f'Is the hypothesis "Entailment", "Neutral", or "Contradiction" of the premise?\n'
    pre_prompt += "Given the context, which of the options best answers the question?\n"

    pre_prompt += 'Rephrase and expand the question, and provide answer.\nAnswer:'
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format Answer={answer}.\n"
    # pre_prompt += "Let’s reiterate the question and also think step by step. Then provide the answer in the format 999 {answer} 999.\n"

    # pre_prompt += "The answer is: "
    return pre_prompt

In [96]:
print(construct_prompt(cqa, 3, 0))

You are a smart assistant. 
It kind of threw me , I did n't expect to get a job that quick . It paid well too , so Wednesday morning I was in the city landscaping . That meant move sand all day .

What can be said of my education level ?
Options:
1. None of the above choices .
2. I probably have decent level of education like college .
3. I probably do n't have much education beyond the minimum .
4. I probably have much education like an advanced degree .
Answer: 3
That breakfast should have more than filled me up . I think my body is already starting to get into the routine of having a midmorning snack , something I did n't really do before school started . I had some grapes . And some leftover gnocchi .

What will they likely eat for lunch ?
Options:
1. Cafeteria food
2. Olive garden
3. McDonalds
4. None of the above choices .
Answer: 1
Then I took the bus home sitting next to a fat old dude with the worst breath ever and he could n't stop talking to some other old guy about stuff an

In [97]:
l = 2
prompt = construct_prompt(cqa, 3, l)
reply = get_reply(prompt)
print(reply)
print(cqa['validation'][l])
# print(labels[snli['test'][l]['label']])

You are a smart assistant. 
Throughout the whole time , nobody in the bus station or the phone service could tell us when the bus was going to get there . We ended up roaming around downtown UTSA and got food at McDonalds . Then we got soooo lost that Anthony had to take I-35 to the direction heading to Austin , so we could find our way back to the bus station . We drove on literally the wrong side of the street several times ( turning to one - way streets ) , it was so dangerous , but everybody was safe .

what may have happened if they had gotten efficient information when they called either the bus station or the phone service ?
Options:
1. They would have been made aware of the bus ' schedule
2. They would have roamed around downtown UTSA
3. They would have been lost
4. None of the above choices .
Answer: 1
In my sister 's garage . My sister was hiding a Porsche from me . I put one hand on my hip . " Do you KNOW , " I said sternly , waving the diaper in the air with my other hand ,

In [98]:
total_count = 0
total_correct = 0
wrong_ids = []
wrong_answers = []
ground_truths = []
for k in range(500):
    
    label = cqa['validation'][k]['label']
    
    if label == -1:
        continue
    
    groundtruth = str(label+1)

    total_count += 1
    print("k: ", k)
    prompt = construct_prompt(cqa, 3, k)
    reply = get_reply(prompt)
    split_reply = reply.split("Answer:")
    answer = split_reply[4].strip().split(" ")[0].lower()

    if groundtruth in answer:
        print("Yes:")
        total_correct += 1
    else:
        wrong_ids.append(k)
        wrong_answers.append(answer)
        ground_truths.append(groundtruth)
        print("N")
        print("Reply: ", answer)
        print("GT: ", groundtruth)
    if k % 10 == 0:
        print(f"{total_correct}/{total_count} = {total_correct/total_count}")

k:  0
N
Reply:  3
GT:  2
0/1 = 0.0
k:  1
N
Reply:  3
GT:  1
k:  2
Yes:
k:  3
Yes:
k:  4
Yes:
k:  5
N
Reply:  4.
GT:  1
k:  6
N
Reply:  3
GT:  1
k:  7
N
Reply:  3
GT:  1
k:  8
Yes:
k:  9
N
Reply:  3
i
GT:  1
k:  10
N
Reply:  2
GT:  4
4/11 = 0.36363636363636365
k:  11
Yes:
k:  12
N
Reply:  3
i've
GT:  1
k:  13
Yes:
k:  14
N
Reply:  2

i
GT:  1
k:  15
Yes:
k:  16
Yes:
k:  17
Yes:
k:  18
N
Reply:  3
i
GT:  2
k:  19
Yes:
k:  20
N
Reply:  3
GT:  1
10/21 = 0.47619047619047616
k:  21
Yes:
k:  22
Yes:
k:  23
Yes:
k:  24
N
Reply:  3
GT:  2
k:  25
Yes:
k:  26
N
Reply:  2
GT:  4
k:  27
Yes:
k:  28
N
Reply:  3
GT:  4
k:  29
Yes:
k:  30
N
Reply:  2
GT:  4
16/31 = 0.5161290322580645
k:  31
N
Reply:  3
GT:  2
k:  32
N
Reply:  4
i
GT:  1
k:  33
Yes:
k:  34
N
Reply:  3
GT:  4
k:  35
Yes:
k:  36
Yes:
k:  37
Yes:
k:  38
N
Reply:  2
GT:  1
k:  39
Yes:
k:  40
Yes:
22/41 = 0.5365853658536586
k:  41
Yes:
k:  42
N
Reply:  4.
GT:  2
k:  43
N
Reply:  4
GT:  3
k:  44
N
Reply:  4
GT:  2
k:  45
N
Reply:  3
GT:  2
k

In [99]:
print(f"{total_correct}/{total_count} = {total_correct/total_count}")

249/500 = 0.498
